In [1]:
# --------------------------------------------------
# 1️⃣ Mount Google Drive (for cache)
# --------------------------------------------------
from google.colab import drive, output
import os
if not os.path.ismount("/content/drive"):
    drive.mount("/content/drive")

# ==================================================
# 2️⃣ Repo & paths
# ==================================================
from pathlib import Path
import subprocess, sys

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

def run(cmd):
    #print("   $", " ".join(map(str, cmd)))
    subprocess.run(cmd, check=True)

# --------------------------------------------------
# 3️⃣ Clone repository (idempotent)
# --------------------------------------------------
if not REPO_DIR.exists():
    print("📥 Cloning fenicsx-colab...")
    run(["git", "clone", REPO_URL, str(REPO_DIR)])
elif not (REPO_DIR / ".git").exists():
    raise RuntimeError("Directory exists but is not a git repository")
else:
    print("📦 Repo already exists — skipping clone")

# --------------------------------------------------
# 4️⃣ Run setup in current kernel
#    Option: add '--clean' to force reinstall
# ------------------------------
USE_CLEAN = False  # <--- Set True to remove existing environment
opts = "--clean" if USE_CLEAN else ""

# Run the setup script
get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts}"
)

# ==================================================
# 5️⃣ Verify %%fenicsx magic is registered
# ==================================================
try:
    get_ipython().run_cell_magic('fenicsx', '--info -np 4', '')
except Exception as e:
    print("⚠️ %%fenicsx magic not found:", e)

Mounted at /content/drive
📥 Cloning fenicsx-colab...
🔧 Installing FEniCSx environment...
✨ Loading FEniCSx Jupyter magic...

🐍 Python          : 3.11.14
📦 dolfinx         : 0.10.0
💻 Platform        : Linux-6.6.105+-x86_64-with-glibc2.35
🧵 Running as root : True

🔎 fenicsx runtime info
-----------------------
Environment        : fenicsx
micromamba         : /content/micromamba/bin/micromamba
MPI implementation : OPENMPI
MPI version        : mpiexec (OpenRTE) 4.1.2
MPI ranks (-np)    : 4


In [2]:
%%fenicsx -np 4 --time

from mpi4py import MPI
import dolfinx

comm = MPI.COMM_WORLD
print(f"Hello from rank {comm.rank}", flush=True)
if comm.rank == 0:
    print(f"  dolfinx : {dolfinx.__version__}")
    print(f"  MPI size: {comm.size}")

Hello from rank 0
Hello from rank 2
Hello from rank 3
Hello from rank 1
  dolfinx : 0.10.0
  MPI size: 4
⏱ Elapsed time: 1.035985 s
